In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os, tempfile, glob, random, json
from pathlib import Path
from IPython.display import Markdown
from PIL import Image
from getpass import getpass
import numpy as np
from itertools import combinations
import tiktoken
# Embedding model
from FlagEmbedding import BGEM3FlagModel

# LLM: openai and google_genai
import openai
from langchain_openai import OpenAI, OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# LLM: HuggingFace
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.llms import HuggingFaceHub

# langchain prompts, memory, chains...
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.documents import Document
from langchain_core.prompts.base import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string

# Document loaders
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    DirectoryLoader,
    CSVLoader,
    UnstructuredExcelLoader,
    Docx2txtLoader,
    JSONLoader
)

# Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter

# OutputParser
from langchain_core.output_parsers import StrOutputParser

# Chroma: vectorstore
from langchain_community.vectorstores import Chroma

# Contextual Compression

#from langchain.retrievers import DocumentCompressorPipeline

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter,LongContextReorder

#from langchain.retrievers import EmbeddingsFilter
#from langchain.retrievers import ContextualCompressionRetriever

# Cohere
from langchain_community.llms import Cohere

### Cargamos el JSON CPEP

In [2]:
with open("../resources/cpe.json", "r", encoding="utf-8") as f:
    data = json.load(f)

docs = []

In [3]:
for item in data:
    tipo = item.get("tipo", "").capitalize()

    if tipo == "Introduccion":
        text = f"Título: {item.get('titulo', '')}\n" \
                f"Subtítulo: {item.get('subtitulo', '')}\n\n" \
                f"{item.get('contenido', '')}"

    elif tipo == "Articulo":
        text = (
            f"Parte {item.get('parte_num', '')}: {item.get('parte_nom', '')}\n"
            f"Título {item.get('titulo_num', '')}: {item.get('titulo_nom', '')}\n"
            f"Capítulo {item.get('capitulo_num', '')}: {item.get('capitulo_nom', '')}\n"
            f"Sección {item.get('seccion_num', '')}: {item.get('seccion_nom', '')}\n"
            f"Artículo {item.get('art_num', '')}: {item.get('nombre_juridico', '')}\n\n"
            f"{item.get('contenido', '')}"
        )
        
    elif tipo == "Disposición":
        text = (
            f"Disposición {item.get('disposicion', '')}\n"
            f"Nombre jurídico: {item.get('nombre_juridico', '')}\n\n"
            f"{item.get('contenido', '')}"
        )

    # Pongo este else, porque sino me sale error en page_content=text, porque es posible que text este vacio
    else:
        text = item.get("contenido", "")

    docs.append(Document(page_content=text, metadata={"tipo": tipo}))

print(f"Se cargaron {len(docs)} documentos desde el JSON.")

Se cargaron 426 documentos desde el JSON.


In [4]:
import random
random_document_id = random.choice(range(len(docs)))

print("test: ", random_document_id)
print(docs[random_document_id])

test:  84
page_content='Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 2: Derechos Fundamentales y Garantías
Capítulo Sexto: Educación, interculturalidad y derechos culturales
Sección 1: Educación
Artículo 83: Participación Social, Comunitaria y de los Padres de Familia en la Educación

Se reconoce y garantiza la participación social, la participación comunitaria y de los padres de familia en el sistema educativo, mediante organismos representativos en todos los niveles del Estado y en las naciones y pueblos indígena originario campesinos. Su composición y atribuciones estarán establecidas en la ley.' metadata={'tipo': 'Articulo'}


In [5]:
# Embedding Model
model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
def tiktoken_tokens(documents)